<a href="https://colab.research.google.com/github/Vardhan-R/LLMs-Impersonate-Chat-Users/blob/main/LLM_Google_Chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Impersonates Google Chat Users

In [ ]:
from google.ai.generativelanguage_v1beta.types import content
from IPython.display import display
from IPython.display import Markdown
import base64
import codecs
import datetime
import google.generativeai as genai
import json
import os
import re

In [ ]:
# Parse the arguments

model = "gemini-1.5-flash"  # @param {isTemplate: true}
contents_b64 = b'W3sicGFydHMiOiBbeyJ0ZXh0IjogIkhlbGxvIn1dfV0='
generation_config_b64 = "e30="  # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# print(json.dumps(contents, indent=4))

In [ ]:
os.environ["GEMINI_API_KEY"] = "GEMINI_API_KEY"

In [ ]:
class Message:
    def __init__(self, text: str, sender: str, dt: datetime.datetime) -> None:
        self.text = text
        self.sender = sender
        self.datetime = dt  # UTC
        self.uts = dt.timestamp()

def getGeminiHistory(messages: list[Message]) -> list[dict[str, str]]:
    """
    Convert messages to Gemini history format.

    Parameters:
        messages (list[Message]): List of Message objects.

    Returns:
        list[dict]: List of messages in Gemini history format.
    """

    gemini_history = []

    for msg in messages:
        role = "user" if msg.sender == "Vardhan R" else "model"
        gemini_history.append({
            "role": role,
            "parts": [
                {"text": msg.text}
            ]
        })

    return gemini_history

def getMessages(other_person: str) -> list[Message]:
    datetime_pattern = r"([A-Za-z]+), (\d{1,2}) ([A-Za-z]+) (\d{4}) at (\d{2}):(\d{2}):(\d{2}) (UTC)"
    all_msgs = []

    path_to_file = f"./chats/{other_person}_messages.json"

    with codecs.open(path_to_file, 'r', "utf-8") as fp:
        msgs_str = "".join([line for line in fp])

    d: dict = json.loads(msgs_str)
    msgs: list = d.get("messages")
    for i, msg in enumerate(msgs):
        text = msg.get("text")
        if text:
            sender = msg.get("creator").get("name")
            created_date = msg.get("created_date")
            m = re.match(datetime_pattern, created_date)
            # Extract date and time components
            dd = int(m.group(2))    # Day of the month (DD)
            month_str = m.group(3)  # Month name
            yyyy = int(m.group(4))  # Year (YYYY)
            hh = int(m.group(5))    # Hour (HH)
            mi = int(m.group(6))    # Minute (MM)
            ss = int(m.group(7))    # Second (SS)

            # Convert the month name to a number
            mo = datetime.datetime.strptime(month_str, "%B").month

            # Create a datetime object
            dt = datetime.datetime(yyyy, mo, dd, hh, mi, ss)    # UTC

            all_msgs.append(Message(text.replace('\xa0', ''), sender, dt))

    return all_msgs

In [ ]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [ ]:
other_person = "NAME"
context = getGeminiHistory(getMessages(other_person))
print(len(context))

In [ ]:
chat_session = model.start_chat(
    history=context[520:520 + 156]
)

In [ ]:
while msg := input("You: "):
    response = chat_session.send_message(msg)
    print(f"{other_person}: ", end="")
    display(Markdown(response.text))